<a href="https://colab.research.google.com/github/NHirt32/LLM-News-Digest/blob/dev/API_LLM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Setup

Setting up the environment with required imports and installations


In [1]:
!pip install python-dotenv
!pip install openai

In [94]:
# For API requests
import requests
# For mounting files and securing sensitive data
from google.colab import drive
import os
from dotenv import load_dotenv
# For OpenAI requests
import openai
# For interactive UI
import ipywidgets as widgets
from IPython.display import display

Here I'm mounting a .env file to prevent sharing my API key on my GitHub repository.

In [3]:
drive.mount('/content/drive')
env_path = '/content/drive/MyDrive/API-LLM-Project/.env'
load_dotenv(env_path)
news_key = os.getenv("NEWS_API_KEY")

Mounted at /content/drive


# News API Setup

Setting up API parameters

In [4]:
# Top URLs syntax: url?country={chosen_country}&apiKey=API_KEY
top_headlines_url = 'https://newsapi.org/v2/top-headlines'
temp_country = 'US'
news_params = {
    'country': temp_country,
    'apiKey': news_key
}

Here, we retrieve a response from the News API. If an error occurs, we halt the execution of the notebook; otherwise, we combine the titles and descriptions into a single, readable string to provide to our LLM.

In [5]:
news_response = requests.get(top_headlines_url, params = news_params)
# If the response from News API is not "OK" stop the execution of the project
if news_response.status_code != 200:
  raise RuntimeError(f"Error: {news_response.status_code}, {news_response.reason}")
# Converting request object to dictionary
news_data = news_response.json()
# Now we combine all titles and descriptions into a single legible string
combined_content = '\n\n'.join(
    f"Title: {article['title']} \n Description{article['description']}"
    for article in news_data['articles']
)

# OpenAI API Setup

Here we setup our OpenAI API parameters.

In [91]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai_params = {
    'model': 'gpt-4o',
    'messages': [
        {"role": "system", "content": "You are an intelligent summarization assistant tasked with breaking down news articles into distinct categories based on their content. Your goal is to analyze the articles and categorize them into the following segments with the appropriate percentage distribution. List a maximum of 3 articles in each section ranked by the article's importance. Ensure that each category is clearly defined and summarized concisely. Use headings or bullet points for clarity. Do not repeat the same article in multiple categories. Do not consider interplanetary news to be environmental. If there are no articles in a category the % breakdown should be 0% for that article."},
        {"role": "user", "content": f"Analyze the following articles and categorize them as Political, Economic, Social Issues, Environment, Technology, and Entertainment: {combined_content}"}

    ],
    # Response length variable
    'max_tokens': 1000
}

Now using our API parameters we make a request to OpenAI to summarize the combined content of our News API data.

In [92]:
try:
  openai_response = openai.chat.completions.create(**openai_params)
  # article_summary = openai_response.choices[0].message.content.strip()
  # print(f"Summary of Articles: {article_summary}")
except Exception as e:
  print(f"An error occured: {e}")

In [93]:
article_summary = openai_response.choices[0].message.content.strip()
print(f"{article_summary}")

**Political - 30%**

1. **Iran's enemies will receive crushing response - Khamenei - BBC.com**
   - This article addresses international relations and potential military conflict following an attack on Iran by Israel.

2. **Trump wants someone who will ‘do his bidding’ at DOJ. Here are the top contenders. - POLITICO**
   - This discusses political maneuvering and potential changes in the Department of Justice depending on future presidential outcomes.

3. **Kemi Badenoch becomes first Black woman to lead UK Conservatives - Reuters UK**
   - This discusses a significant political development in the United Kingdom with implications for political leadership and policy direction.

**Economic - 20%**

1. **Berkshire's cash soars to $325 billion, Buffett sells Apple, Bank of America - Reuters**
   - An analysis of Berkshire Hathaway's financial strategies impacting major economic stakeholders and markets.

2. **TGI Fridays files for bankruptcy, COVID-19 pandemic blamed for financial woes - F

In [39]:
for i in range(0, 10):
  print(news_data['articles'][i]['title'])

Ohio State remains in Big Ten title mix, tops PSU - ESPN
Iran's enemies will receive crushing response - Khamenei - BBC.com
'RHOBH' alum Teddi Mellencamp files for divorce from husband Edwin Arroyave after 13 years of marriage - Page Six
Florida's convicted killer clown released from prison for the murder of her husband's then-wife - The Associated Press
Trump wants someone who will ‘do his bidding’ at DOJ. Here are the top contenders. - POLITICO
[Removed]
Alan Rachins, Actor on ‘L.A. Law’ and ‘Dharma & Greg,’ Dies at 82 - Hollywood Reporter
Yankees decline 2025 option on Anthony Rizzo, Gerrit Cole opts out of deal - Yahoo Sports
Kemi Badenoch becomes first Black woman to lead UK Conservatives - Reuters UK
Bubba Watson Donates House After Making Ace at Indonesian Masters - Sports Illustrated


In [95]:
country_dropdown = widgets.Dropdown(
    options=[
        ('United States', 'us'),
        ('United Kingdom', 'gb'),
        ('Canada', 'ca'),
        ('Australia', 'au'),
        ('Germany', 'de'),
        ('France', 'fr'),
        ('India', 'in'),
    ],
    value='us',  # default value
    description='Country:',
)
display(country_dropdown)


Dropdown(description='Country:', options=(('United States', 'us'), ('United Kingdom', 'gb'), ('Canada', 'ca'),…